In [ ]:
import yfinance as yf
import finnhub
import mysql.connector
import schedule
import time

finnhub_client = finnhub.Client(api_key="cv6loq9r01qi7f6qvhqgcv6loq9r01qi7f6qvhr0")  

def fetch_and_store_stock_data():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="#Falguniak1979",
            database="market_data"
        )
        cursor = conn.cursor()

        symbol = "AAPL"

        quote = finnhub_client.quote(symbol)

        stock = yf.Ticker(symbol)
        volume = int(stock.history(period="1d")["Volume"].iloc[-1])  

        cursor.execute("""
            INSERT INTO real_time_stocks (symbol, open, high, low, current_price, previous_close, volume)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            symbol,
            quote.get("o", 0),  
            quote.get("h", 0),  
            quote.get("l", 0),  
            quote.get("c", 0),  
            quote.get("pc", 0),  
            volume  
        ))

        conn.commit()
        conn.close()
        print(f"✅ Data updated for {symbol} at {time.strftime('%H:%M:%S')}")

    except Exception as e:
        print(f"⚠️ Error: {e}")

schedule.every(5).minutes.do(fetch_and_store_stock_data)

print("⏳ Market data collection started... Press CTRL+C to stop.")
while True:
    schedule.run_pending()
    time.sleep(1)  